# Trabalho #1 - Biblioteca de dados, RNAs pré-treinadas e PipeLine de dados

Nesse trabalho você vai criar e treinar uma RNA, usando como base uma rede pré-treinada, para resolver um problema de classificação multiclasse.

Para fazer isso você vai criar um pipeline de dados para treinamento, usar os vetores de características da RNA EfficientNet e um conjunto de dados de imagens de satélite para treinar uma nova RNA para classificar tipos de áreas a partir de imagens de satélites.


## Coloque o seu nome aqui:

Nome:

Nome:

Nome:

## 1. Importar bibliotecas

Primeiramente devemos instalar o tf_keras, que é uma versão compatível do Keras com o TensorFlow 2.17 para permitir usar alguns modelos do TensorFlow Hub.

Execute as células abaixo para importar as bilbiotecas necessárias.

In [ ]:
!pip install tf_keras

In [ ]:
import tensorflow as tf
import tf_keras as keras
print("Using TensorFlow Version:", tf.__version__)
print("Using Keras Version:", keras.__version__)

import tensorflow_datasets as tfds
import tensorflow_hub as hub

import matplotlib.pyplot as plt
import numpy as np

## 2. Carregar dados do TensorFlow Data Service (TFDS)

O conjunto de dados EuroSAT é baseado nas imagens do satélite Sentinel-2 e consiste de 27.000 imagens coloridas de dimensão 64x64x3 com 10 classes.  

Dois conjuntos de dados são disponibilizados: (1) eurosat/rgb que contém imagens no formato RGB, que é o que usaremos; e (2) eurosat/all com imagens de 13 canais diferentes.


### Exercício #1: Carregar dados

A primeira etapa é carregar as imagens. Esse conjunto de dados está disponível no TFDS com o nome `eurosat/rgb`. Os dados estão em um único conjunto de nome "train" e, portanto precisam ser divididos em pelo menos dois conjuntos: dados de treinamento e de validação.

Na célula abaixo inclua o seu código para carregar esse conjunto de dados. Mais detalhes de como carregar esse dados podem ser vistos em https://www.tensorflow.org/datasets.

Ao carregar os dados, use o argumento `split` com porcentagens para separar os dados em dois conjuntos: dados de treinamento (80% dos dados) e de validação (20% dos dados). Para obter maiores detalhes de como usar o  método `tfds.load` pode ser obtido em https://www.tensorflow.org/datasets/splits.

In [ ]:
# Carrega dados do TFDS
# Inclua seu código aqui (2 linhas)
train_data, info = tfds.load('eurosat/rgb', shuffle_files=True, as_supervised=True, with_info=True, split="train[:80%]")
val_data = tfds.load('eurosat/rgb', as_supervised=True, split="train[80%:]")

Execute a célula abaixo para visualizar as informações sobre esse o conjunto de dados `eurosat/rgb`.

In [ ]:
info

Verifique o número de exemplos de treinamento e de validação executando a célula abaixo.

In [ ]:
print('Número exemplos de treinamento =', len(list(train_data)))
print('Número exemplos de treinamento =', len(list(val_data)))

**Saída esperada:**

    Número exemplos de treinamento = 21600
    Número exemplos de treinamento = 5400

Execute a célula abaixo para definir a lista com os nomes das classes existentes no conjutno de dados.

In [ ]:
labels_list = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial',
               'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
print(labels_list)

### Exercíco #2: Visualização dos dados

Na célula abaixo escreva um código para obter 5 exemplos dos dados de treinamento e visualizá-los juntamento com os nomes (`labels_list`) e números das classes.

Para fazer isso você vai precisar de um laço `for`, dos comandos `print`, `plt.imshow()`, `plt.show()`, e do método `take()`.

In [ ]:
# Itera no conjunto de dados pegando exemplos
# Inclua seu código aqui (~5 linhas)
for data in train_data.take(3):
    image, label = data
    print("Classe:", labels_list[label], '-', format(label))
    plt.imshow(image)
    plt.show()

**Exemplo de saída esperada (sem as figuras):**

    Classe: Forest - 1
    Classe: Residential - 7
    Classe: River - 8
    


## 3. Processamento dos dados

Após carregar os dados, você tem que processá-los para poderem ser usados pela RNA.

O módulo de vetores de características da EfficientNet permite que as imagens tenham em princípio qualquer dimensão, assim, não é necessário redimensionar as imagens. Porém, os valores dos pixels devem estar entre 0 e 1, conforme mencionado nas instruções de uso desse módulo, que podem ser vistas em https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1.

### Exercício #3: Normalização das imagens

Na célula abaixo escreva um código para normalizar um lote de imagens e depois crie os lotes de dados de treinamento e de validação.

Conforme vimos na aula, ao importar os dados do TF Data Services, os dados são armazenados em objetos e para podermos usar esses dados de forma eficiente temos que usar os métodos fornecidos para esse tipo de objeto.

Para normalizar as imagens crie uma função de nome `format_image` e a utilize chamando o método `map()`. Para normalizar os dados você vai precisar primeiramente transformá-los em `float32`, para isso use a função `tf.cast()`. As instruções de uso dessa função podem ser vistas em https://www.tensorflow.org/api_docs/python/tf/cast.

Crie um dataset eficiente, para isso utilize processamento em paralelo quando executar a função de pré-processamento dos dados. Use tamném os métodos `cache` e `prefetch`.

In [ ]:
# Definição da dimensão das imagens para processamento e do tamanho dos lotes de dados

# Use otimização do pipeline
AUTOTUNE = tf.data.AUTOTUNE

# Inclua seu código aqui (~2 linhas)
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 32

# Função usada para redimensionar e normalizar as imagens
# Inclua seu código aqui (~3 linhas)
def format_image(image, label):
    image = tf.cast(image, dtype=tf.float32) / 255.0
    return  image, label

# Cria lotes de dados usando o método map() para chamar a função format_image()
# Inclua seu código aqui (~2 linhas)
train_batches = train_data.map(format_image, num_parallel_calls=AUTOTUNE).cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
val_batches = val_data.map(format_image, num_parallel_calls=AUTOTUNE).cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

train_batches

**Saída espeada:**

    <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>  

Execute a célula abixo para verificar se o seu dataset está correto.

In [ ]:
for img, label in train_batches.take(1):
    print('Dimensão de um lote de imagens:', img.shape)
    print('Dimensão de um lote de saídas desejadas:', label.shape)

**Saída esperada:**

    Dimensão de um lote de imagens: (32, 64, 64, 3)
    Dimensão de um lote de saídas desejadas: (32,)

## 4.  Criação da RNA

Nesse trabalho você vai criar e treinar uma RNA para identificar tipos de áreas a partir de imagens do satélite Sentinel-2.

Para isso você vai criar uma RNA usando como base o módulo de vetor de características da rede EfficientNet, que foi treinada com as imagens da ImageNet.

As redes EfficientNets são utilizadas para classificar imagens e apresentam um desempenho similar a outras redes mais conhecidas, porém, possui um número muito menor de parâmetros e é muito mais rápida. O trabalho que originou essa RNA é
Mingxing Tan and Quoc V. Le: EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks, ICML 2019.

### Exercício #4: Carregar vetor de características do TF-Hub

Na célula abaixo crie um código que importa o módulo de vetores de características da EfficientNet e o coloca no objeto `feature_extractor`. Essa rede estão no TF-Hub e as informações de como usá-la podem ser obtidas no link https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1.

Pra fazer isso você vai precisar definir a dimensão do tensor de entrada da rede usando o argumento `input_shape` e a função `hub.KerasLayer()`.

In [ ]:
# Dimensão das imagens para argumento input_shape
# Inclua seu código aqui (~1 linha)
IMAGE_DIM = (64, 64, 3)

# Carrega vetores de características com a URL do módulo
# Inclua seu código aqui (~2 linhas)
MODULE_HANDLE ="https://tfhub.dev/google/efficientnet/b0/feature-vector/1"
feature_extractor = hub.KerasLayer(MODULE_HANDLE, input_shape=IMAGE_DIM)
feature_extractor

**Saída espeada:**

    <tensorflow_hub.keras_layer.KerasLayer at 0x7ff58df7ae48>

In [ ]:
for img, label in train_batches.take(2):
    feat = feature_extractor(img)
    print(feat.shape)

**Saída esperada:**

    (32, 1280)
    (32, 1280)

### Exercício #5: Criação da RNA como o Keras

Na célula abaixo crie um código que incorpora o `feature_extractor`, criado no exercício #4, em uma rede sequencial do Keras para realizar a tarefa de classificação multiclasse com 10 classes.

Após criar a RNA utilize o método `summary()` para apresentá-la.

In [ ]:
# Número de classes da RNA
# Inclua seu código aqui (~1 linha)
NUM_CLASSES = 10

# Cria modelo seguencial do Keras para problema de classificação com 10 classes
# Inclua seu código aqui (~3 linhas)
rna = keras.Sequential([
       keras.layers.InputLayer(input_shape=IMAGE_DIM),
       feature_extractor,
       keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Apresenta configuração da RNA
# Inclua seu código aqui (~1 linha)
rna.summary()

**Saída esperada:**

    Model: "sequential"
    _________________________________________________________________
    Layer (type)                 Output Shape              Param #   
    =================================================================
    keras_layer (KerasLayer)     (None, 1280)              4049564   
    _________________________________________________________________
    dense (Dense)                (None, 10)                12810     
    =================================================================
    Total params: 4,062,374
    Trainable params: 12,810
    Non-trainable params: 4,049,564
    _________________________________________________________________

## 5. Compilação e treinamento da RNA

Como visto em aula, o treinamento da RNA deve ser realizado de forma que somente os parâmetros da camada densa, adicionada ao extrator de características, sejam alterados durante o treinamento. Isso é necessário para não destrui a parte da RNA que corresponde à EfficientNet, que já foi previamente treinada com um conjunto de centenas de milhares de imagens. Assim, você tem que "congelar" os parâmetros do extrator e características.

### Exercício #6: Compilação da RNA

Na célula abaixo crie um código que congela os parâmetros do `feature_extractor` e compila a RNA usando os seguintes parâmetros:

- Método de otimização: Adam;
- Fução de custo: `sparse_categorical_crossentropy`;
- Métrica: `accuracy`.

In [ ]:
# Congela parâmetros da MobiliNet
# Inclua seu código aqui (~1 linha)
feature_extractor.trainable = False

# Define método de otimização
# Inclua seu código aqui (~1 linha)
optimizer = 'adam'

# Compila RNA
# Inclua seu código aqui (~1 comando)
rna.compile(optimizer=optimizer,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

### Exercício #7: Treinamento da RNA

O treinamento da RNA deve ser realizado com o método `fit` e os dados de treinamento e validação são fornecidos por meio dos objetos `train_batches` e `val_batches`.

Na célula abaixo crie um código que realiza o treinameto da RNA usando 30 épocas de treinamento.

In [ ]:
# Define número de épocas de treinamento
# Inclua seu código aqui (~1 linha)
EPOCHS = 30

# Realiza o treinamento usando os dados de treinamento e validação
# Inclua seu código aqui (~1 comando)
history = rna.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=val_batches)

**Saída esperada:**

    Epoch 1/10
    675/675 [==============================] - 13s 19ms/step - loss: 0.5030 - accuracy: 0.8542 - val_loss: 0.3120 - val_accuracy: 0.9002
    .
    .
    .
    Epoch 30/30
    675/675 [==============================] - 8s 11ms/step - loss: 0.0695 - accuracy: 0.9830 - val_loss: 0.2688 - val_accuracy: 0.9181

### Exercício #8: Resultados do treinamento

Na célula abaixo crie um código que apresenta os resultados do treinamento em função das épocas. Você deve fazer dois gráficos:

1. Valores da função de custo para os dados de treinamento e de validação;
2. Valores da métrica para os dados de treinamento e de validação.

In [ ]:
# Definir vetores com valores da função de custo e da métrica para os dados de treinamento e de validação
# Inclua seu código aqui (~5 linhas)
history_dict = history.history
custo = history_dict['loss']
exatidao = history_dict['accuracy']
custo_val = history_dict['val_loss']
exatidao_val = history_dict['val_accuracy']

# Criar vetor de épocas
# Inclua seu código aqui (~1 linha)
epocas = range(1, len(custo) + 1)

# Fazer o gráfico dos valores da função de custo
# Inclua seu código aqui (~6 linhas)
plt.plot(epocas, custo, 'b')
plt.plot(epocas, custo_val, 'r')
plt.title('Valor da função de custo')
plt.xlabel('Épocas')
plt.ylabel('Custo')
plt.grid()
plt.show()


# Fazer o gráfico dos valores da métrica
# Inclua seu código aqui (~6 linhas)
plt.plot(epocas, exatidao, 'b')
plt.plot(epocas, exatidao_val, 'r')
plt.title('Valor da exatidão')
plt.xlabel('Épocas')
plt.ylabel('Erro médio')
plt.grid()
plt.show()


## 6.4 Teste da RNA

Após o treinamento é necessário verificar o desempenho da RNA. paar isso vamos calcular os valores da função de custo e da métrica para as imagens do conjunto de validação e depois vamos usar o método `predict` para prever as classes de algumas imagens.

O código da célula abaixo calcula o resultado da função de custo e da exatidão para os exemplos validação usando o método `evaluate`.

### Exercício #9: Avaliação do desempenho da RNA

Na célula abaixo determine o desemepnho da RNA usando o método `evaluate` para calcular o valor da função d ecusto e da métrica para os dados de validação.

In [ ]:
# Avalia desempenho da RNA para os dados de validação
# Inclua seu código aqui (~1 linha)
eval_results = rna.evaluate(val_batches, verbose=0)

# Apresenta os resultados
# Inclua seu código aqui (~2 linhas)
for metric, value in zip(rna.metrics_names, eval_results):
    print(metric + ': {:.4}'.format(value))

**Saída esperada:**

    loss: 0.2684
    accuracy: 0.9181

### Exercício #10: Teste de classificação de imagens

Para poder fornecer as imagens para a RNA usando o método `predict` você precisa extrai-las do objeto `val_data` e processá-las com a função `format_image`, que por sua vez é chamada pelo método `map()`. Além disso, você tem que incluir o eixo dos exemplos na imagem de acordo com o esperado por uma RNA do Keras.

Na célula baixo crie um código que calcula as classes previstas para os 5 primeiros exemplos do conjunto de validação usando o método `predict` e apresenta os resultados junto com as imagens e as classes previstas e reais.

In [ ]:
# Itera no objeto val_data para pegar 5 imagens e aplica função format_image
# Inclua seu código aqui (~1 linha)
for data in val_data.map(format_image).take(5):

    # Extrai imagem e classe prevista
    # Inclua seu código aqui (~1 linha)
    image, label = data

    # Adiciona eixo dos exemplos
    # Inclua seu código aqui (~1 linha)
    image = np.expand_dims(image, axis=0)

    # Calcula probabilidades previstas pela RNA
    # Inclua seu código aqui (~1 linha)
    yprev = rna.predict(image)

    # Determina classe prevista
    # Inclua seu código aqui (~1 linha)
    label_prev = np.argmax(yprev)

    # Apresenta resultados das classes e mostra imagem
    # Inclua seu código aqui (~4 linhas)
    print('Classe prevista =', labels_list[label_prev], ', Classe real =', labels_list[label])
    plt.imshow(image[0])
    plt.show()

**Saída esperada (sem incluir as imagens):**

    Classe prevista = Industrial , Classe real = Industrial
    Classe prevista = AnnualCrop , Classe real = AnnualCrop
    Classe prevista = HerbaceousVegetation , Classe real = HerbaceousVegetation
    Classe prevista = River , Classe real = River
    Classe prevista = Residential , Classe real = Residential